In [ ]:
import os
import sys
import subprocess
import Bio
from Bio.PDB import PDBParser, PDBIO


In [1]:
pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
parser = PDBParser()
enzyme_structure = parser.get_structure('enzyme', '4dfr.pdb')



In [ ]:
try:
    with open('ligand.json', 'r') as f:
        ligand_structure = parser.get_structure('ligand','ligand.json')
except FileNotFoundError:
    print('Error: ligand.json not found.')
    sys.exit(1)


In [ ]:
# Prepare the enzyme and ligand files for molecular docking
enzyme_file = 'enzyme.pdbqt'
ligand_file = 'ligand.pdbqt'
io = PDBIO()
io.set_structure(enzyme_structure)
io.save(enzyme_file)
io = PDBIO()
io.set_structure(ligand_structure)
io.save(ligand_file)

In [ ]:

# Run molecular docking using Autodock Vina
vina_exe = 'vina' #location

docking_output = 'docking.pdbqt'
docking_cmd = f'{vina_exe} --receptor {enzyme_file} --ligand {ligand_file} --out {docking_output} --num_modes 10'
subprocess.run(docking_cmd, shell=True)


In [ ]:

# Parse the docking output file to find the best binding pose
docking_pose = None
with open(docking_output, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line.startswith('MODEL'):
            docking_pose = line.split()[1]



In [ ]:
# Find the residues within 5A of the best binding pose
groove_residues = []
for residue in enzyme_structure.get_residues():
    ligand_atoms = []
    for atom in ligand_structure.get_atoms():
        if float(atom['occupancy']) == 1.0 and float(atom['bfactor']) == 0.0:
            ligand_atoms.append(atom)
    ligand_center = Bio.PDB.geometry.center_of_mass(ligand_atoms)
    distance = residue.get_distance(ligand_center)
    if distance < 5.0:
        groove_residues.append(residue)



In [ ]:
# Print the groove residues
for residue in groove_residues:
    print(residue.get_resname(), residue.get_id())